In [1]:
import numpy as np
import pandas as pd

from glob import glob
from os import path

In [2]:
ABCD_SSD_dists = pd.read_csv('abcd_data/SSD_dist_by_subj.csv')
ABCD_SSD_dists

,NARGUID,SSDDur,proportion
0,003RTV85,300.0,0.250000
1,003RTV85,250.0,0.183333
2,003RTV85,400.0,0.183333
3,003RTV85,350.0,0.166667
4,003RTV85,450.0,0.100000
...,...,...,...
62441,xxmy9wd8,100.0,0.183333
62442,xxmy9wd8,150.0,0.133333
62443,xxmy9wd8,200.0,0.116667
62444,xxmy9wd8,250.0,0.100000


In [3]:
indiv_SSRTs = pd.DataFrame()
for file in [i for i in glob('ssrt_metrics/indiv_*.csv') if 'linear' not in i]:
    data =  pd.read_csv(file, index_col=0)
    
    if 'graded' in file:
        data.loc[data.SSD==0, ['SSRT', 'SSRT_w_guesses', 'SSRT_w_graded']] = data.loc[data.SSD==50.0, ['SSRT', 'SSRT_w_guesses', 'SSRT_w_graded']].values
    else:
        data.loc[data.SSD==0, 'SSRT_w_graded'] = data.loc[data.SSD==50.0, 'SSRT_w_graded'].values
    
    gen_str = file.split('indiv_')[-1].replace('.csv', '')
    indiv_ssrt_df = pd.DataFrame(np.nan, index=ABCD_SSD_dists.NARGUID.unique(), columns=['vanilla', 'w_guesses', 'w_graded'])
    for sub in ABCD_SSD_dists.NARGUID.unique():
        curr_sub = ABCD_SSD_dists.query("NARGUID=='%s'" % sub)
        indiv_ssrt_df.loc[sub, :] = [0, 0, 0]
        for SSD in curr_sub.SSDDur:
            ssd_SSRTs = data.loc[data.SSD==SSD, ['SSRT', 'SSRT_w_guesses', 'SSRT_w_graded']].values[0]
            weight = curr_sub.loc[curr_sub.SSDDur==SSD, 'proportion'].values
            indiv_ssrt_df.loc[sub, :] +=  ssd_SSRTs * weight
    indiv_SSRTs = pd.concat([indiv_SSRTs, indiv_ssrt_df.add_prefix('gen-%s_SSRT-' % gen_str)], 1)

In [4]:
indiv_SSRTs.to_csv('ABCD_weighted_SSRTs.csv')
indiv_SSRTs

,gen-vanilla_SSRT-vanilla,gen-vanilla_SSRT-w_guesses,gen-vanilla_SSRT-w_graded,gen-guesses_SSRT-vanilla,gen-guesses_SSRT-w_guesses,gen-guesses_SSRT-w_graded,gen-graded_mu_go_log_SSRT-vanilla,gen-graded_mu_go_log_SSRT-w_guesses,gen-graded_mu_go_log_SSRT-w_graded
003RTV85,284.416667,267.664180,392.533333,298.250000,284.379822,409.933333,221.366667,195.565274,294.566667
00CY2MDM,323.150000,173.950794,1013.283333,417.016667,294.242667,1211.133333,220.700000,41.882793,410.466667
00HEV6HB,308.716667,223.993990,751.650000,366.116667,290.760294,919.566667,217.600000,114.959070,355.916667
00LJVZK2,301.283333,232.592216,681.683333,349.000000,287.702491,797.050000,228.483333,141.776528,346.433333
00NPMHND,275.633333,272.341594,327.116667,280.566667,277.513156,332.283333,253.683333,246.047601,282.650000
...,...,...,...,...,...,...,...,...,...
hkfm1ruj,286.533333,256.471343,453.550000,308.083333,283.535095,490.266667,229.816667,189.624792,305.883333
hx1ru4hv,308.800000,209.858004,725.066667,372.916667,293.278372,844.100000,217.683333,85.602629,360.533333
jf8w3pw6,304.450000,213.137570,647.583333,362.350000,292.902792,708.133333,217.683333,92.541857,351.100000
x8k59,299.316667,229.002230,572.600000,345.483333,291.506226,623.200000,217.100000,116.936086,335.300000


In [5]:
indiv_SSRTs.corr().to_csv('ABCD_weighted_SSRTs_corr.csv')
indiv_SSRTs.corr()

,gen-vanilla_SSRT-vanilla,gen-vanilla_SSRT-w_guesses,gen-vanilla_SSRT-w_graded,gen-guesses_SSRT-vanilla,gen-guesses_SSRT-w_guesses,gen-guesses_SSRT-w_graded,gen-graded_mu_go_log_SSRT-vanilla,gen-graded_mu_go_log_SSRT-w_guesses,gen-graded_mu_go_log_SSRT-w_graded
gen-vanilla_SSRT-vanilla,1.000000,-0.932260,0.975765,0.995826,0.869396,0.967271,-0.473648,-0.942552,0.986896
gen-vanilla_SSRT-w_guesses,-0.932260,1.000000,-0.878685,-0.959884,-0.815143,-0.869575,0.469482,0.973896,-0.894590
gen-vanilla_SSRT-w_graded,0.975765,-0.878685,1.000000,0.966654,0.752222,0.997619,-0.311040,-0.857231,0.993537
gen-guesses_SSRT-vanilla,0.995826,-0.959884,0.966654,1.000000,0.863212,0.957375,-0.460295,-0.958905,0.979719
gen-guesses_SSRT-w_guesses,0.869396,-0.815143,0.752222,0.863212,1.000000,0.725770,-0.755974,-0.921629,0.807407
gen-guesses_SSRT-w_graded,0.967271,-0.869575,0.997619,0.957375,0.725770,1.000000,-0.293854,-0.842049,0.984234
gen-graded_mu_go_log_SSRT-vanilla,-0.473648,0.469482,-0.311040,-0.460295,-0.755974,-0.293854,1.000000,0.629002,-0.354825
gen-graded_mu_go_log_SSRT-w_guesses,-0.942552,0.973896,-0.857231,-0.958905,-0.921629,-0.842049,0.629002,1.000000,-0.887804
gen-graded_mu_go_log_SSRT-w_graded,0.986896,-0.894590,0.993537,0.979719,0.807407,0.984234,-0.354825,-0.887804,1.000000


In [6]:
indiv_SSRTs_rank = pd.DataFrame()
for col in indiv_SSRTs.columns:
    indiv_SSRTs_rank[col] = indiv_SSRTs[col].rank()
indiv_SSRTs_rank.corr().to_csv('ABCD_weighted_SSRTs_rank-corr.csv')
indiv_SSRTs_rank.corr()

,gen-vanilla_SSRT-vanilla,gen-vanilla_SSRT-w_guesses,gen-vanilla_SSRT-w_graded,gen-guesses_SSRT-vanilla,gen-guesses_SSRT-w_guesses,gen-guesses_SSRT-w_graded,gen-graded_mu_go_log_SSRT-vanilla,gen-graded_mu_go_log_SSRT-w_guesses,gen-graded_mu_go_log_SSRT-w_graded
gen-vanilla_SSRT-vanilla,1.000000,-0.985927,0.987652,0.998724,0.968190,0.985753,-0.455835,-0.993443,0.992408
gen-vanilla_SSRT-w_guesses,-0.985927,1.000000,-0.976196,-0.992188,-0.953891,-0.972729,0.404623,0.986143,-0.983362
gen-vanilla_SSRT-w_graded,0.987652,-0.976196,1.000000,0.986844,0.932068,0.998366,-0.392233,-0.969201,0.996863
gen-guesses_SSRT-vanilla,0.998724,-0.992188,0.986844,1.000000,0.966630,0.984704,-0.443090,-0.993959,0.992270
gen-guesses_SSRT-w_guesses,0.968190,-0.953891,0.932068,0.966630,1.000000,0.926324,-0.553837,-0.983624,0.949465
gen-guesses_SSRT-w_graded,0.985753,-0.972729,0.998366,0.984704,0.926324,1.000000,-0.393137,-0.965292,0.992773
gen-graded_mu_go_log_SSRT-vanilla,-0.455835,0.404623,-0.392233,-0.443090,-0.553837,-0.393137,1.000000,0.482958,-0.409314
gen-graded_mu_go_log_SSRT-w_guesses,-0.993443,0.986143,-0.969201,-0.993959,-0.983624,-0.965292,0.482958,1.000000,-0.979443
gen-graded_mu_go_log_SSRT-w_graded,0.992408,-0.983362,0.996863,0.992270,0.949465,0.992773,-0.409314,-0.979443,1.000000


In [10]:
indiv_SSRTs_rank.filter(regex='SSRT-vanilla').corr()

,gen-vanilla_SSRT-vanilla,gen-guesses_SSRT-vanilla,gen-graded_mu_go_log_SSRT-vanilla
gen-vanilla_SSRT-vanilla,1.000000,0.998724,-0.455835
gen-guesses_SSRT-vanilla,0.998724,1.000000,-0.443090
gen-graded_mu_go_log_SSRT-vanilla,-0.455835,-0.443090,1.000000
